# Learning outcomes
1. white balancing techniques
2. Edge enhancement techniques (Image sharpening)

# White balancing

White balancing remains a critical preprocessing step in computer vision applications, ensuring color consistency regardless of illumination conditions. OpenCV's contrib packages, particularly the xphoto module offers sophisticated algorithms for automatic white balancing. 

## Intro to white balance
White balance refers to the process of removing unrealistic color casts from images, ensuring the objects appearing white in person are rendered white in captured images regardless of lighting conditions. Digital sensors, unlike the human visual system lack the adaptive capability to automatically compensate for varying illuminants. 

The fundamental concept behind white balancing involves estimating the color of the illuminant in a scene and then compensating for it by adjusting color channel intensities. White balancing is typically executed as a two-step process: first estimating the scene illuminant and then applying a correction to neutralize its effect on the image colors.

In Python, the functionality can be accessed through the `cv2.xphoto` module. 
```
wb = cv.createGrayWorldWB()   # for gray world algorithm
wb = cv.createSimpleWB()   # for simple white balance
wb = cv.createLearningBasedWB()   # for learning-based approach
``` 

## simple white balance algorithm
The SimpleWB class implements a straightforward white balance algorithm that operates by independently stretching each color channel of the input image to a specified range. This approach is based on the assumption that each color channel should utilize the full dynamic range in a properly white-balanced image.

The implementation offers several configurable parameters that control its behavior:
1. InputMin and InputMax: Defining the expected range of input pixel values.
2. OutputMin and OutputMax: Specifying the desired range for output pixel values.
3. P parameter: controlling the percentage of top and bottom pixel values to ignore, which increases the robustness against outliers.

In [5]:
import cv2 as cv
from utils import display_images

# create wb class
wb = cv.xphoto.createSimpleWB()

# load image
img = cv.imread('./images/alley_night.jpg')

result = wb.balanceWhite(img)

# create another wb class with different P param
wb1 = cv.xphoto.createSimpleWB()
wb1.setP(0.05)  # ignore 5% of top and bottom pixels

result1 = wb1.balanceWhite(img)

display_images([img, result, result1], 
               ["image", "white balanced", "wb (0.05)"])

In [10]:
print("The expected input min-max of the input image: ", (wb.getInputMin(), wb.getInputMax()))
print("The output ranges of the input image: ", (wb.getOutputMin(), wb.getOutputMax()))

The expected input min-max of the input image:  (0.0, 255.0)
The output ranges of the input image:  (0.0, 255.0)


## Gray world white balance algorithm
The GrayworldWB class implements the well established gray-world assumption algorithm, which postulates that under a neutral illuminant, the average of all colors in an image should be gray. This method assumes that, in a sufficiently varied scene, the average reflectance of surfaces is achromatic (i.e., has equal RGB components).

It adds a modification which thresholds pixels based on their saturation value and only uses pixels below the provided threshold in finding average pixel values.

Saturation is calculated using the following for a 3-channel RGB image per pixel I and is in the range [0, 1]:
$$Saturation[I] = \frac{max(R,G,B) - min(R,G,B)}{max(R,G,B)}$$

A threshold of 1 means that all pixels are used to white-balance.

In [6]:
# gray world assumption white balancing
wb_gray_world = cv.xphoto.createGrayworldWB()

result = wb_gray_world.balanceWhite(img)

display_images([img, result], ("Image", "gray world white balance"))

In [3]:
wb_gray_world = cv.xphoto.createGrayworldWB()

img = cv.imread("./images/night_view.jpg")

result = wb_gray_world.balanceWhite(img)

display_images([img, result], ("Image", "gray world white balance"))

# Image sharpening techniques

## Unsharp masking
Unsharp masking is a linear image processing technique used to sharpen images by enhancing their perceived sharpness and clarity. In the context of digital images, unsharp masking works by subtracting a blurred version of the image from the image itself to create an "unsharp mask", which emphasizes edges and fine details.

The math formula:
$$V = x + \gamma (x-y)$$

Where:
- $x$ is original input image.
- $y$ is the blurred version of the input image, obtained using low-pass filter.
- $\gamma$ is a scaling factor that controls the strength of the sharpening effect.
- $V$ is the resulting sharpened image.

Alternatively, in some advanced implementations like Photoshop's Unsharp Mask, the formula involves additional steps for refinement:
$$O_{sharpened} = O + (O - GB) - inv(O + inv(GB))$$

Here:
- $O$ is the original image
- $GB$ represents the Gaussian-blurred version of the image.
- $inv(.)$ denotes inversion operations. In the context of image processing, inversion typically involves flipping pixel intensity values.

In [12]:
# unsharp masking operation
import numpy as np

def unsharp_masking(img: np.ndarray, scaling_factor: float, ksize: int=5) -> np.ndarray:
    """Unsharp masking
    Args:
        img: (numpy array), input image
        scaling_factor: float, sharpening effect
        ksize: int, kernel size of blur filter
    Returns:
        Resulting image (numpy array)"""
    blur = cv.blur(img, (ksize, ksize))
    img_float = img.astype(np.float32)
    edge_details = scaling_factor * (img_float - blur)
    return cv.convertScaleAbs(img_float + edge_details)

In [13]:
img = cv.imread("./images/car.jpg")

enhanced_img = unsharp_masking(img, scaling_factor=1.0)

display_images([img, enhanced_img], ("image", "unsharp masking"))

In [14]:
# inverse
def inv(img: np.ndarray) -> np.ndarray:
    """Inversion of image: 255 - x"""
    return 255 - img

# advanced unsharp masking
def unsharp_masking_v2(img: np.ndarray, ksize: int=5):
    """Advanced unsharp masking
    Args:
        img: numpy array, input image
        ksize: int, default Gaussian kernel size=5
    Returns:
        output image (numpy array)"""
    blurred = cv.GaussianBlur(img, (ksize, ksize), 0)

    img_float = img.astype(np.float32)
    term3 = inv(img_float + inv(blurred))
    term2 = img_float - blurred
    sharpened = img_float + term2 - term3
    return cv.convertScaleAbs(sharpened)

In [16]:
img = cv.imread("./images/night_view.jpg")

enhanced = unsharp_masking_v2(img)

display_images([img, enhanced], ("original", "enhanced"))

## Image sharpening using Laplacian
Steps:
1. Apply the Laplacian filter: Convolve the original image with the Laplacian kernel to produce a filtered image that emphasizes edges.
2. Subtract the Laplacian from the original image: Subtract the Laplacian-filtered image from the original image. This step enhances edges by amplifying the differences in intensity:
$$g(x, y) = f(x,y) - \nabla^2f(x,y)$$
Here, $g(x,y)$ is the sharpened image, and $f(x, y)$ is the original image.
3. Adjust for negative values: Since the Laplacian operator can produce negative values, the result is often scaled or adjusted to ensure all pixel values remain within the valid range.

In [17]:
# sharpening using Laplacian
def sharpening_Laplacian(img: np.ndarray) -> np.ndarray:
    """Image sharpening using Laplacian filter
    Args:
        img: numpy array, source
    Returns:
        numpy array, destination image"""
    # convert the image to grayscale
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # apply Laplacian filter
    laplacian = cv.Laplacian(gray, cv.CV_64F)
    laplacian_abs = cv.convertScaleAbs(laplacian)

    # combine the original gray image and the Laplacian
    enhanced = cv.addWeighted(gray, 1.0, laplacian_abs, 1.0, 0)
    return enhanced

In [18]:
img = cv.imread("./images/meal.jpg")

enhanced = sharpening_Laplacian(img)

display_images([img, enhanced], ("original image", "Laplacian filter"))

## Edge enhancement in color image
1. Convert to a suitable color space (optional): While you can work directly in RGB, converting the image to a color space like Lab (L for lightness, a and b for color components) can be advantageous.
2. Detect edges using a color edge detection technique: Instead of converting to grayscale and losing color-specific edge info, use a method that accounts for all channels. A powerful approach is the Di Zenzo structure tensor method, which computes the gradient across all color channels:
    * For each channel (R, G, B), calculate the spatial gradients (e.g., using Sobel operators) in the x and y directions: $G_{xR}, G_{yR}, G_{xG}, G_{yG}, G_{xB}, G_{yB}$.
    * Form the structure tensor by summing the outer products of these gradients over the channels.
    * Compute the eigenvalues of the tensor; the largest eigenvalue at each pixel represents the edge strength, giving a single edge map $E$ that captures both intensity and color edges.
3. Extract the detail layer: To enhance edges, isolate the high-frequency components (details, including edges) of the image:
    * For each channel $c(R, G, B)$, apply a Gaussian blur to create a smoothed version $c_{blurred}$.
    * Compute the detail layer for each channel: $D_c = c - c_{blurred}$. This represents the edges and fine details.
4. Enhance image using the edge map: use the edge map, $E$ to guide the sharpening process:
    * For each channel, compute the sharpened version: $c_{sharpened} = c + \lambda.E.D_c$, where $\lambda$ is a scaling factor to control the enhancement strength.
    * The edge map E ensures that sharpening is stronger where edges are prominent and weaker in smooth areas, preventing over-enhancement of noise or float regions.
5. Combine the channels.

In [23]:
def enhance_edges_color(img: np.ndarray, sigma: float=2.0, lambda_=1.0):
    """Enhance edges in a color image using structure tensor and detail layer amplication.
    
    Args:
        - img: Input color image (BGR)
        - sigma: standard deviation for gaussian blur (default: 2.0)
        - lambda_: enhancement strength factor (default: 1.0)
        
    Returns:
        - enhanced_image: edge enhanced color image"""
    # split the image into B, G, R channels
    B, G, R = cv.split(img)
    B = B.astype(np.float32)
    G = G.astype(np.float32)
    R = R.astype(np.float32)

    # compute Sobel gradients for each channels (x and y directions)
    G_xB = cv.Sobel(B, cv.CV_32F, 1, 0, ksize=3)
    G_yB = cv.Sobel(B, cv.CV_32F, 0, 1, ksize=3)
    G_xG = cv.Sobel(G, cv.CV_32F, 1, 0, ksize=3)
    G_yG = cv.Sobel(G, cv.CV_32F, 0, 1, ksize=3)
    G_xR = cv.Sobel(R, cv.CV_32F, 1, 0, ksize=3)
    G_yR = cv.Sobel(R, cv.CV_32F, 0, 1, ksize=3)

    # compute structure tensor components
    J_11 = G_xB ** 2 + G_xG ** 2 + G_xR ** 2
    J_12 = G_xB * G_yB + G_xG * G_yG + G_xR * G_yR
    J_22 = G_yB ** 2 + G_yG ** 2 + G_yR ** 2

    # compute the largest eigenvalue
    discriminant = (J_11 - J_22) ** 2 + 4 * J_12 ** 2
    lambda_max = 0.5 * (J_11 + J_22 + np.sqrt(np.maximum(discriminant, 0)))

    # normalize the edge map
    E_max = np.max(lambda_max)
    if E_max > 0:
        E = lambda_max / E_max
    else:
        E = lambda_max

    # compute the blurred versions of each channel to extract details
    B_blurred = cv.GaussianBlur(B, (0, 0), sigmaX=sigma)
    G_blurred = cv.GaussianBlur(G, (0, 0), sigmaX=sigma)
    R_blurred = cv.GaussianBlur(R, (0, 0), sigmaX=sigma)

    # compute detail layers
    D_B = B - B_blurred
    D_G = G - G_blurred
    D_R = R - R_blurred

    # enhance each channel using edge map
    B_sharpened = B + lambda_ * E * D_B
    G_sharpened = G + lambda_ * E * D_G
    R_sharpened = R + lambda_ * E * D_R

    # clip values to [0, 255]
    B_sharpened = np.clip(B_sharpened, 0, 255).astype(np.uint8)
    G_sharpened = np.clip(G_sharpened, 0, 255).astype(np.uint8)
    R_sharpened = np.clip(R_sharpened, 0, 255).astype(np.uint8)

    enhanced_image = cv.merge([B_sharpened, G_sharpened, R_sharpened])

    return enhanced_image

In [32]:
img = cv.imread("./images/book_page.jpg")

enhanced = enhance_edges_color(img, lambda_=2.0)

display_images([img, enhanced], ("original", "Enhanced"))